In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np

In [2]:
import os
import cv2

In [17]:
import pandas as pd

In [3]:
sources = ["https://towardsdatascience.com/build-your-own-convolution-neural-network-in-5-mins-4217c2cf964f"]

In [174]:
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

In [176]:
def CNN(num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(H,W,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adadelta',metrics=['accuracy'], options=run_opts)
    return model

In [147]:
image_root = 'dataset/jaffe/'
image_title = os.listdir(image_root)
image_dirs = [image_root + x for x in os.listdir(image_root)]

In [167]:
H = 192
W = 192
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    dimension = (H,W)
    resized = cv2.resize(img, dimension)
    return resized
def prep_data(images):
    count = len(images)
    data = np.ndarray((count, 1, H, W), dtype=np.uint8)
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image
        if i%50 == 0: print('Processed {} of {}'.format(i, count))
    return data

In [168]:
images = prep_data(image_dirs)

Processed 0 of 213
Processed 50 of 213
Processed 100 of 213
Processed 150 of 213
Processed 200 of 213


In [161]:
image_labels = np.array([title[3:5] for title in image_title])

In [162]:
labels = []
for i, label in enumerate(image_labels):
    if label == 'AN':
        labels.append(0)
    if label == 'DI':
        labels.append(1)
    if label == 'FE':
        labels.append(2)
    if label == 'SU':
        labels.append(3)
    if label == 'SA':
        labels.append(4)
    if label == 'HA':
        labels.append(5)
    if label == 'NE':
        labels.append(6)

In [169]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
 
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, random_state=69)

X_train = X_train.reshape(X_train.shape[0], H, W, 1)
X_test = X_test.reshape(X_test.shape[0], H, W, 1)

In [177]:
cnn_model = CNN(3)

In [ ]:
cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10,batch_size=8, verbose=2)

Train on 170 samples, validate on 43 samples
Epoch 1/10


In [166]:
scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

Classification Error: 90.70%
